In [1]:
import gurobipy as gp

In [12]:
# Parâmetros do problema
qtd_itens = 63
qtd_mochilas = 3

# Vetores com os pesos, valores e capacidades
vet_pesos = [145, 159, 148, 128, 80, 92, 141, 178, 93, 146, 64, 129, 83, 84, 127, 168, 75, 80, 153, 133, 110, 79, 105, 87, 87, 53, 158, 146, 51, 107, 116, 165, 167, 134, 169, 62, 57, 127, 54, 199, 115, 108, 138, 81, 75, 169, 148, 191, 82, 123, 189, 189, 146, 55, 140, 88, 154, 157, 106, 53, 192, 180, 177]
vet_valores = [326, 417, 333, 345, 202, 209, 363, 404, 232, 350, 172, 309, 213, 201, 311, 424, 174, 204, 397, 302, 253, 215, 270, 230, 221, 125, 430, 397, 132, 272, 281, 400, 375, 314, 447, 150, 128, 339, 139, 542, 290, 251, 317, 212, 168, 452, 399, 453, 223, 319, 505, 448, 365, 127, 318, 200, 400, 365, 262, 137, 441, 423, 469]
vet_capacidades = [1500, 2000, 1700]

# Rótulos dos itens
itens = list()
for i in range(qtd_itens):
    itens.append("Item_{}".format(i + 1))

# Rótulos das mochilas
mochilas = list()
for j in range(qtd_mochilas):
    mochilas.append("Mochila_{}".format(j + 1))

# Dicionário das capacidades
capacidades = dict()
for idx, cap in enumerate(vet_capacidades):
    rotulo = mochilas[idx]
    capacidades[rotulo] = cap
    
# Dicionários dos pesos e valores
pesos = dict()
valores = dict()
for i in range(qtd_itens):
    rotulo = itens[i]
    pesos[rotulo] = vet_pesos[i]
    valores[rotulo] = vet_valores[i]

In [18]:
m = gp.Model()
m.setParam(gp.GRB.Param.OutputFlag, 0)

# Insere as variáveis de decisão
x = m.addVars(itens, mochilas, vtype=gp.GRB.BINARY)

# Define a função objetivo
m.setObjective(
    gp.quicksum(x[i, j] * valores[i] for i in itens for j in mochilas),
    sense=gp.GRB.MAXIMIZE)

# Restrição 1
c1 = m.addConstrs(
    gp.quicksum(x[i, j] for j in mochilas) <= 1 for i in itens)

# Restrição 2
c2 = m.addConstrs(
    gp.quicksum(x[i, j] * pesos[i] for i in itens) <= capacidades[j] for j in mochilas)

In [21]:
# Executa o modelo
m.optimize()

# Valor objetivo
print("Valor Total:", m.objVal)

# Mostra os itens nas mochilas
for j in mochilas:
    print(j)
    for i in itens:
        if round(x[i, j].X) == 1:
            print(i, " ", end="")
    print("\n")

Valor Total: 13364.0
Mochila_1
Item_7  Item_9  Item_10  Item_13  Item_14  Item_15  Item_29  Item_30  Item_31  Item_50  Item_53  Item_59  Item_63  

Mochila_2
Item_5  Item_16  Item_18  Item_19  Item_22  Item_23  Item_24  Item_25  Item_27  Item_28  Item_39  Item_41  Item_46  Item_47  Item_48  Item_62  

Mochila_3
Item_2  Item_4  Item_11  Item_12  Item_32  Item_35  Item_38  Item_40  Item_44  Item_49  Item_51  Item_57  Item_60  



In [22]:
for j in mochilas:
    print(j, "Capacidade utilizada:", capacidades[j] - c2[j].Slack)

Mochila_1 Capacidade utilizada: 1500.0
Mochila_2 Capacidade utilizada: 2000.0
Mochila_3 Capacidade utilizada: 1699.0
